In [1]:
!wget https://www.dropbox.com/s/ttc9hmcca04lc02/sal_data.zip?dl=0
!mv sal_data.zip?dl=0 sal_data.zip
!unzip sal_data.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!wget https://www.dropbox.com/s/0yluib4kmbhi9f4/sal_train_val.zip?dl=0
!mv sal_train_val.zip?dl=0 sal_train_val.zip
!unzip sal_train_val.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip install pydot
!pip install pyemd

  Using cached pydot-1.4.2-py2.py3-none-any.whl (21 kB)
  Created wheel for pyemd: filename=pyemd-0.5.1-cp36-cp36m-win_amd64.whl size=69200 sha256=4fee366578fd576743ccaf41a2d86d9d0359e5fc4ac49bf34384e24e4ae8a13e
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\f9\f0\23\aefbdde40e915c67830ebecb55be2344a8b6e95fe3ce3ccf96
Successfully built pyemd


In [4]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import keras
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Concatenate,Activation
from keras.layers import concatenate
from keras.layers import Conv2D, Conv2DTranspose, ZeroPadding2D,MaxPooling2D, Cropping2D, BatchNormalization
from keras.models import Model
from keras import metrics
from keras import backend as K
from keras import optimizers
from keras import losses
from keras.utils import plot_model
from keras.callbacks import Callback
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import tensorflow as tf
import random
import time
import numpy as np
import skimage.io as io
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pydot
from pyemd import emd, emd_samples

Using TensorFlow backend.
c:\Users\User\anaconda3\envs\summer_placement\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\User\anaconda3\envs\summer_placement\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\User\anaconda3\envs\summer_placement\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\User\ana

In [6]:
K.set_floatx('float32')
emd_arr = []
lcc_arr = []

In [7]:
class TestCallback(Callback):
    def __init__(self, X_train, Y_train, model):
      self.X_train = X_train
      self.Y_train = np.array(Y_train)
      self.model = model

      
    #'Called at the end of the epoch'     
    def on_epoch_end(self, epoch, logs={}):
        global emd_arr, lcc_arr
        [y_p, y_t] = self.get_y(20)
        emd_val = self.get_emd(y_p, y_t)
        lcc_val = self.get_lcc(y_p, y_t)
        print('EMD', emd_val)
        print('LCC', lcc_val)
        emd_arr.append(emd_val)
        lcc_arr.append(lcc_val)
      
      
    #'Returns the prediction and the true value in numpy array format'    
    def get_y(self, index):
        y_pred = self.model.predict(self.X_train)
        y_pred = np.array(y_pred[3])
        y_true = np.array(self.Y_train)
        y_p = y_pred[index,:,:,0]
        y_t = y_true[index,:,:,0]
        y_p = np.ravel(y_p)
        y_t = np.ravel(y_t)
        return [y_p, y_t]
      
      
    #'Returns the Earth Movers Distance between prediction and truth'
    def get_emd(self,y_pred, y_true):
      return emd_samples(y_pred, y_true)
    
    
    #'Returns the Linear Correlation Coefficient between prediction and truth '
    def get_lcc(self,y_pred, y_true):
      return np.corrcoef(y_pred, y_true)[0,1]
    
 

In [8]:
class AttentionModel:
  def __init__(self):
        self.batch_size = 16
        self.epochs = 2 #10
        self.lr = 0.0001
        self.train_no_images = 100#0
        self.test_no_images = 10#0
        
        
  #'Returns Data arrays with given number of samples'      
  def dataLoadBatch(self, num_samples):
        X=[]
        Y=[]
        for h in range(0, num_samples):
            I = io.imread("data/images/salMap_{:05d}.jpg".format(h))
            X.append(I)
            bin_label = np.zeros((224,224))
            labels = io.imread("data/salMap/salMap_{:05d}.jpg".format(h))[:,:,0]
            for i in range(0,224):
                for j in range(0,224):
                  if labels[i][j]<26:
                      bin_label[i][j] = 0
                  elif labels[i][j]<51:
                      bin_label[i][j] = 0.111
                  elif labels[i][j]<76:
                      bin_label[i][j] = 0.222
                  elif labels[i][j]<102:
                      bin_label[i][j] = 0.333
                  elif labels[i][j]<128:
                      bin_label[i][j] = 0.444
                  elif labels[i][j]<154:
                      bin_label[i][j] = 0.556
                  elif labels[i][j]<180:
                      bin_label[i][j] = 0.667
                  elif labels[i][j]<206:
                      bin_label[i][j] = 0.778
                  elif labels[i][j]<230:
                      bin_label[i][j] = 0.889                  
                  else:
                      bin_label[i][j] = 1
            Y.append(bin_label)
        X = np.array(X)
        Y = np.array(Y)
        
        return X,Y
        
        
  #Defines the model architecture      
  def DeepAttentionModel(self):
    img_rows, img_cols, img_chns = 224, 224, 3
    if K.image_data_format() == 'channels_first':
      original_img_size = (img_chns, img_rows, img_cols)
    else:
      original_img_size = (img_rows, img_cols, img_chns)
    
    
    self.x = Input(shape=original_img_size)
    padded_x = ZeroPadding2D(padding=(35), data_format="channels_last")(self.x)
    
    
    #'Encoder'
    conv_1_1 = Conv2D(64, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_x)
    padded_conv_1_1 = ZeroPadding2D(padding=(1), data_format="channels_last")(conv_1_1)
    conv_1_2 = Conv2D(64, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_conv_1_1)
    pool_1 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format="channels_last")(conv_1_2)
    
    padded_input_pool_1=  ZeroPadding2D(padding=(1), data_format="channels_last")(pool_1)
    conv_2_1 = Conv2D(128, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_input_pool_1)
    padded_input_conv_2_1 = ZeroPadding2D(padding=(1), data_format="channels_last")(conv_2_1)
    conv_2_2 = Conv2D(128, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_input_conv_2_1)
    pool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format="channels_last")(conv_2_2)
    
    padded_input_pool_2=  ZeroPadding2D(padding=(1), data_format=None)(pool_2)
    conv_3_1 = Conv2D(256, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_pool_2)
    padded_input_conv_3_1 = ZeroPadding2D(padding=(1), data_format=None)(conv_3_1)
    conv_3_2 = Conv2D(256, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_3_1)
    padded_input_conv_3_2 = ZeroPadding2D(padding=(1), data_format=None)(conv_3_2)
    conv_3_3 = Conv2D(256, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_3_2)
    pool_3 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format=None)(conv_3_3)
    
    padded_input_pool_3 =  ZeroPadding2D(padding=(1), data_format=None)(pool_3)
    conv_4_1 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_pool_3)
    padded_input_conv_4_1 = ZeroPadding2D(padding=(1), data_format=None)(conv_4_1)
    conv_4_2 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_4_1)
    padded_input_conv_4_2 = ZeroPadding2D(padding=(1), data_format=None)(conv_4_2)
    conv_4_3 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_4_2)
    pool_4 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format=None)(conv_4_3)
    
    padded_input_pool_4 =  ZeroPadding2D(padding=(1), data_format=None)(pool_4)
    conv_5_1 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_pool_4)
    padded_input_conv_5_1 = ZeroPadding2D(padding=(1), data_format=None)(conv_5_1)
    conv_5_2 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_5_1)
    padded_input_conv_5_2 = ZeroPadding2D(padding=(1), data_format=None)(conv_5_2)
    conv_5_3 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_5_2)
    
    
    #'Decoder'
    deconv_5_1 = Conv2DTranspose(512,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(conv_5_3)
    deconv_5_2 = Conv2DTranspose(256,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_5_1)
    deconv_5_3 = Conv2DTranspose(128,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_5_2)
    deconv_5_4 = Conv2DTranspose(64,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_5_3)
    
    attention1 = Conv2D(1, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(deconv_5_4 )
    attention1c = Cropping2D(cropping= ((54,54),(54,54)), data_format=None)(attention1)
    self.bn_attention1c = BatchNormalization(name='ba1c')(attention1c)
    
    deconv_4_1 = Conv2DTranspose(256,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(conv_4_3)
    deconv_4_2 = Conv2DTranspose(128,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_4_1)
    deconv_4_3 = Conv2DTranspose(64,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_4_2)
    
    attention2 = Conv2D(1, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(deconv_4_3 )
    attention2c = Cropping2D(cropping= ((42,42),(42,42)))(attention2)
    self.bn_attention2c = BatchNormalization(name='ba2c')(attention2c)
    
    deconv_3_1 = Conv2DTranspose(128,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(conv_3_3)
    deconv_3_2 = Conv2DTranspose(64,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_3_1)
    
    attention3 = Conv2D(1, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(deconv_3_2)
    attention3c = Cropping2D(cropping= ((36,36),(36,36)))(attention3)
    self.bn_attention3c = BatchNormalization(name='ba3c')(attention3c)
    
    attention = concatenate([attention1c,attention2c,attention3c])
    padded_attention = ZeroPadding2D(padding=(1))(attention)
    final_attention =  Conv2D(1, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_attention)
    self.bn_final_attention = BatchNormalization(name='bafc')(final_attention)
    
    
    self.model = Model(inputs=self.x, outputs=[self.bn_attention1c,self.bn_attention2c,self.bn_attention3c,self.bn_final_attention ])
    
    def custom_loss(y_true, y_pred):
             loss1=losses.binary_crossentropy(y_true,self.bn_attention1c)
             loss2=losses.binary_crossentropy(y_true,self.bn_attention2c)
             loss3=losses.binary_crossentropy(y_true,self.bn_attention3c)
             loss4=losses.binary_crossentropy(y_true,self.bn_final_attention)
             return (loss1+loss2+loss3+loss4)/4.0
          
    sgd = optimizers.SGD(lr=self.lr) #Stochastic Gradient Descent Optimizer
    self.loss = custom_loss
    self.model.compile(optimizer = sgd , loss = self.loss, metrics=['accuracy'])
    
    
    
  def Train(self):  
    
    # Load data
    X, Y= self.dataLoadBatch(self.train_no_images + self.test_no_images)
    
    # Train and Test split
    X_train = X[0:self.train_no_images,:,:,:]
    Y_train = Y[0:self.train_no_images,:,:,np.newaxis]
    X_test =  X[self.train_no_images : self.train_no_images + self.test_no_images,  : , : , :]
    Y_test =  Y[self.train_no_images : self.train_no_images + self.test_no_images,  : , :,np.newaxis]
    
    # History object stores loss and accuracy
    hist_obj = self.model.fit(X_train, [Y_train,Y_train,Y_train,Y_train],validation_data=(X_test,[Y_test,Y_test,Y_test,Y_test]), \
                   batch_size=self.batch_size, epochs=self.epochs, callbacks=[TestCallback(X_train, Y_train, self.model)])       
    output = self.model.predict(X_test)
    self.output = np.array(output)
    
    # Making data global, to be used for visualization
    self.input_image_full = X[:,:,:]
    self.req_output_image_full =Y[:,:,:]
    
    return hist_obj
 
  
  # Prints the model architecture
  def get_Model_Summary(self):      
      print(self.model.summary())
    

  def Visualise_Output(self,idx): 
      self.input_image =self.input_image_full[self.train_no_images +idx,:,:]
      self.req_output_image =self.req_output_image_full[self.train_no_images +idx,:,:]
      pred_image_1 = self.output[0,idx,:,:,:]
      pred_image_2 = self.output[1,idx,:,:,:]
      pred_image_3 = self.output[2,idx,:,:,:]
      pred_image_4 = self.output[3,idx,:,:,:]
      fig = plt.figure(figsize=(20,10))
      plt.subplot(231)
      plt.imshow((self.input_image.reshape(224,224,3)), interpolation='none')
      plt.title("Input - ")
      ax = plt.gca()
      ax.grid(False)
      plt.subplot(232)
      plt.imshow(self.req_output_image.reshape(224,224)*255,interpolation='none')
      plt.title("Ground Truth Attention")
      ax = plt.gca()
      ax.grid(False)
      plt.subplot(233)
      plt.imshow(pred_image_1.reshape(224,224)*255,interpolation='none')
      plt.title("Predicted Attention 1 ")
      ax = plt.gca()
      ax.grid(False)
      plt.subplot(234)
      plt.imshow(pred_image_2.reshape(224,224)*255,interpolation='none')
      plt.title("Predicted Attention 2 ")
      ax = plt.gca()
      ax.grid(False)
      plt.subplot(235)
      plt.imshow(pred_image_3.reshape(224,224)*255,interpolation='none')
      plt.title("Predicted Attention 3 ")
      ax = plt.gca()
      ax.grid(False)
      plt.subplot(236)
      plt.imshow(pred_image_4.reshape(224,224)*255,interpolation='none')
      plt.title("Predicted Final Attention ")
      ax = plt.gca()
      ax.grid(False)
      plot_name = 'plot' + '_'+str(idx)+'.png'
      print(plot_name)
      fig.savefig(plot_name)
    
    
 
  
        
    

In [9]:
deconvNet = AttentionModel()
deconvNet.DeepAttentionModel()

In [10]:
hist_obj = deconvNet.Train()

FileNotFoundError: No such file: 'c:\Users\User\Documents\Github\SaRa\benchmarking\Deep-Visual-Attention-Prediction-master\src\data\images\salMap_00000.jpg'

In [ ]:
train_loss = hist_obj.history['loss']
val_loss = hist_obj.history['val_loss']
train_acc = hist_obj.history['bafc_acc']
val_acc = hist_obj.history['val_bafc_acc']

In [ ]:
fig1 = plt.figure()
plt.plot(train_loss)
plt.plot(val_loss)
plt.legend(['Train','Val'], loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.show()
fig1.savefig('Loss.png')

In [ ]:
fig2 = plt.figure()
plt.plot(train_acc)
plt.plot(val_acc)
plt.legend(['Train','Val'], loc='upper left')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.show()
fig2.savefig('Accuracy.png')


In [ ]:
fig3 = plt.figure()
plt.plot(emd_arr)
plt.xlabel('Epoch')
plt.ylabel('EMD')
plt.title('Earth Movers Distance')
plt.show()
fig3.savefig('EMD.png')

In [ ]:
fig4 = plt.figure()
plt.plot(lcc_arr)
plt.xlabel('Epoch')
plt.ylabel('LCC')
plt.title('Linear Correlation Coefficient')
plt.show()
fig4.savefig('LCC.png')

In [ ]:
index = 1
deconvNet.Visualise_Output(index) 

In [11]:
from google.colab import files
files.download("Loss.png" )
files.download("Accuracy.png" )
files.download("EMD.png")
files.download("LCC.png")
files.download("plot_"+str(index)+".png")

ModuleNotFoundError: No module named 'google.colab'